# Generate daily prices using daily data service - F#

### Overview
Daily prices is designed to illustrate price movements for an instrument over a look back period.

This sample demonstrates how to directly request and plot daily prices.<br>

It enables to retrieve **on-demand** daily prices data by calling a dedicated service on a **daily** data store. 

### Inputs/outputs
Daily prices sample requires instrument's identifier as per input. It returns daily prices and volumes.
This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

### Services used
This sample uses *gRPC requests* in order to retrieve daily bars from the dedicated hosted service. The queried endpoint in this script are:
* *DailyPricesService*: to directly retrieve daily price objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Daily.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run daily prices sample

### Step 1: Install packages

In [1]:
#i "nuget: file://~/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Restore sources file://~/.nuget/packages/ https://api.nuget.org/v3/index.json Installed Packages Systemathics.Apis, 0.13.2-pre XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [2]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Daily.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [3]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *daily prices* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Service called: *DailyPricesService* in order to retrieve daily data

In [4]:
// Set instrument
let ticker = "AAPL"
let exchange = "XNGS"

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [5]:
// Instantiate the daily prices service
let service = new DailyPricesService.DailyPricesServiceClient(channel)

// Create and process prices data request
let request = 
    new DailyPricesRequest(
        Identifier = new Identifier (Exchange = exchange, Ticker = ticker)
    )

let reply = service.DailyPrices(request, headers)

In [6]:
//Visualize request results
reply.Data
|> display

index Date Price Volume Score 0 Year Month Day 2001 1 2 0.2656243625 452257085 0 1 Year Month Day 2001 1 3 0.29241001250000004 816913161 0 2 Year Month Day 2001 1 4 0.3046956973 739252174 0 3 Year Month Day 2001 1 5 0.29241001250000004 412345790 0 4 Year Month Day 2001 1 8 0.2957671473 373470496 0 5 Year Month Day 2001 1 9 0.30692783479999997 588578213 0 6 Year Month Day 2001 1 10 0.2957671473 580715794 0 7 Year Month Day 2001 1 11 0.32142780000000004 803316328 0 8 Year Month Day 2001 1 12 0.30692783479999997 423316216 0 9 Year Month Day 2001 1 16 0.30580283750000004 304836732 0 10 Year Month Day 2001 1 17 0.3002314223 837022809 0 11 Year Month Day 2001 1 18 0.3337134848 1226201343 0 12 Year Month Day 2001 1 19 0.34821345000000004 776525864 0 13 Year Month Day 2001 1 22 0.343749175 519289246 0 14 Year Month Day 2001 1 23 0.36607055 879045310 0 15 Year Month Day 2001 1 24 0.36607055 717059321 0 16 Year Month Day 2001 1 25 0.3560348598 489553175 0 17 Year Month Day 2001 1 26 0.3493384473 482771559 0 18 Year Month Day 2001 1 29 0.3872847848 855295653 0 19 Year Month Day 2001 1 30 0.388391925 692340862 0 (5353 more)

### Step 4: Visualize data

#### 4.1 Plot the prices and volumes
The following code snippets retrive *daily prices* over the look back period from the request result and plot them with the package of your choice.<br><i>`XPlot.Plotly`</i> is used as per open source display package.

In [7]:
// Extract the daily prices and the volumes
let prices = 
    reply.Data
    |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Price))
let volumes = 
    reply.Data
    |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Volume))

In [8]:
// Create the graph
let pricesAndVolumes = 
    [
        new Scattergl(
            name = "Price",
            mode = "lines",
            x = (prices |> Seq.map fst),
            y = (prices |> Seq.map snd)
        )
        new Scattergl(
            name = "Volume",
            mode = "bar",
            x = (volumes |> Seq.map fst),
            y = (volumes |> Seq.map snd),
            yaxis = "y2"
        )
    ]


let layout = 
    new Layout.Layout(
        title = $"Daily price & Volume for {ticker}",
        yaxis = new Yaxis(
            title = "Price", 
            autorange = true,
            side = "left"
        ),
        yaxis2 = new Yaxis(
            title = "Volume", 
            autorange = true,
            side = "right"
        )
    )

Chart.Plot(pricesAndVolumes, layout)
|> Chart.WithXTitle("Time")
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display

<!DOCTYPE html>